# **Exposición**

# Base Diabetes

 ***Integrantes:***

*   Valentina Cañon Cañon
* Juan Pablo Abril Serna
* Sebastian Galvis



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/MachineLearning/expo/datos/diabetes.xlsx'

In [ ]:
df_diabetes = pd.read_excel(file_path)

***Limpieza***

In [ ]:
df_diabetes = df_diabetes.drop_duplicates()

In [ ]:
# Rellenar los valores 0 que no tienen sentido en algunas columnas con la mediana
cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in cols:
    median_value = df_diabetes[col].median()
    df_diabetes[col] = df_diabetes[col].replace(0, median_value)


In [ ]:
# Función para eliminar outliers usando IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Aplicar la eliminación de outliers en las columnas numéricas
numeric_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

for col in numeric_columns:
    df_diabetes = remove_outliers(df_diabetes, col)


In [ ]:
print(f"Tamaño del dataset después de la limpieza: {df_diabetes.shape}")

Tamaño del dataset después de la limpieza: (503, 9)


**División de los datos para entrenamiento y test**

In [ ]:
from sklearn.model_selection import train_test_split

# Separar las características y la variable objetivo
X = df_diabetes.drop('Outcome', axis=1)
y = df_diabetes['Outcome']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


***Árbol de Decisión***

In [ ]:
# Árbol de Decisión
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

# Predicción y evaluación
y_pred_tree = decision_tree.predict(X_test)
print(f'Exactitud del Árbol de Decisión: {accuracy_score(y_test, y_pred_tree):.2f}')

Exactitud del Árbol de Decisión: 0.76


# Implementación de Bagging (Random Forest)

Bagging (Bootstrap Aggregating) crea múltiples modelos de la misma clase (por ejemplo, árboles de decisión) y combina sus predicciones promediando o votando

In [ ]:
# Bagging con Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Predicción y evaluación
y_pred_rf = random_forest.predict(X_test)
print(f'Exactitud de Random Forest (Bagging): {accuracy_score(y_test, y_pred_rf):.2f}')


Exactitud de Random Forest (Bagging): 0.79


 Se observa una mejora en la precisión ya que cada árbol corrige errores locales.


**Nota:**
Bootstrapping es una técnica estadística que consiste en generar múltiples subconjuntos de datos a partir de un conjunto de datos original utilizando muestreo aleatorio con reemplazo
A partir de cada subconjunto (bootstrap sample), se calcula la estadística de interés (por ejemplo, la media, la varianza, etc.).

# Implementación de Boosting (AdaBoost y Gradient Boosting)

Boosting crea modelos secuenciales donde cada nuevo modelo corrige los errores del anterior.

In [ ]:
# AdaBoost
ada_boost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=100, random_state=42)
ada_boost.fit(X_train, y_train)

# Predicción y evaluación
y_pred_ada = ada_boost.predict(X_test)
print(f'Exactitud de AdaBoost: {accuracy_score(y_test, y_pred_ada):.2f}')

# Gradient Boosting
gradient_boost = GradientBoostingClassifier(n_estimators=100, random_state=42)
gradient_boost.fit(X_train, y_train)

# Predicción y evaluación
y_pred_gb = gradient_boost.predict(X_test)
print(f'Exactitud de Gradient Boosting: {accuracy_score(y_test, y_pred_gb):.2f}')



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Exactitud de AdaBoost: 0.76
Exactitud de Gradient Boosting: 0.81


Estos modelos entrenan secuencialmente, donde cada modelo siguiente se enfoca en los errores del modelo anterior, lo que permite una mejora en casos más difíciles de clasificar.

# Implementación de Stacking

Stacking combina diferentes modelos (no necesariamente del mismo tipo) y utiliza un modelo meta para hacer la predicción final.

In [ ]:
# Modelos base para Stacking
estimators = [
    ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42)),
]

# Modelo meta
stacking = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier(max_depth=3))
stacking.fit(X_train, y_train)

# Predicción y evaluación
y_pred_stack = stacking.predict(X_test)
print(f'Exactitud de Stacking: {accuracy_score(y_test, y_pred_stack):.2f}')



Exactitud de Stacking: 0.78


Combina diferentes modelos (Random Forest y Gradient Boosting en este caso) y utiliza un clasificador de nivel superior (árbol de decisión) para mejorar la predicción.

# Comparación de resultados

**Comparación de un solo arbol de desición a Bagging**

In [ ]:
# Exactitud del Bagging (Random Forest) ya calculado
print(f'Exactitud de Random Forest (Bagging): {accuracy_score(y_test, y_pred_rf):.2f}')

# Comparación de resultados
print("Comparación de exactitud:")
print(f'Árbol de Decisión: {accuracy_score(y_test, y_pred_tree):.2f}')
print(f'Random Forest (Bagging): {accuracy_score(y_test, y_pred_rf):.2f}')


Exactitud de Random Forest (Bagging): 0.79
Comparación de exactitud:
Árbol de Decisión: 0.76
Random Forest (Bagging): 0.79


Conclusines

In [ ]:
print("Comparación de exactitud de diferentes técnicas:")
print(f'Random Forest (Bagging): {accuracy_score(y_test, y_pred_rf):.2f}')
print(f'AdaBoost: {accuracy_score(y_test, y_pred_ada):.2f}')
print(f'Gradient Boosting: {accuracy_score(y_test, y_pred_gb):.2f}')
print(f'Stacking: {accuracy_score(y_test, y_pred_stack):.2f}')


Comparación de exactitud de diferentes técnicas:
Random Forest (Bagging): 0.79
AdaBoost: 0.76
Gradient Boosting: 0.81
Stacking: 0.78
